Code pour projet crypto

Import des librairies

**Date limite :** Mercredi 27 mars 2024

**Groupe TD DIA 6 :**

Henri Serano, Sara Thibierge, Eloi Seidlitz

In [137]:
import hashlib
import random 
import gmpy2 as gm
import math
import time
import base64
import os, binascii
from backports.pbkdf2 import pbkdf2_hmac
import pyaes, pbkdf2, secrets


## Exercice 1

1. Trouver une chaîne de caractères (contenant vos noms et prénoms) dont le hash SHA256 se termine par le plus de zéros possible (en hexadécimal).

2. Mesurer le temps moyen pour obtenir n et n+1 zéros en fin de chaîne (n = 5) et calculer le rapport Tn+1/Tn.

### Partie 1

In [138]:

def trouver_chaine(base, zeros_cibles):
        compteur = 0
        while True:
            chaine_test = base + str(compteur)
            hash_resultat = sha256_hash(chaine_test)
            if hash_resultat.endswith('0' * zeros_cibles):
                return chaine_test, compteur, hash_resultat
            compteur += 1

def sha256_hash(chaine):
        return hashlib.sha256(chaine.encode()).hexdigest()
        
def trouver_chaine_et_stats(base, zeros_cibles, iterations_stat=5):
    

    # Première partie : Trouver la chaîne
    chaine_trouvee, compteur_trouve, hash_trouve = trouver_chaine(base, zeros_cibles)

    # Deuxième partie : Statistiques de temps
    temps_n = []
    temps_n_plus_1 = []
    
    for _ in range(iterations_stat):
        debut = time.time()
        trouver_chaine(base, zeros_cibles)
        fin = time.time()
        temps_n.append(fin - debut)

        debut = time.time()
        trouver_chaine(base, zeros_cibles + 1)
        fin = time.time()
        temps_n_plus_1.append(fin - debut)

    temps_moyen_n = sum(temps_n) / len(temps_n)
    temps_moyen_n_plus_1 = sum(temps_n_plus_1) / len(temps_n_plus_1)

    rapport_temps = temps_moyen_n_plus_1 / temps_moyen_n

    return {
        "chaine_trouvee": chaine_trouvee,
        "hash_trouve": hash_trouve,
        "compteur_trouve": compteur_trouve,
        "temps_moyen_n": temps_moyen_n,
        "temps_moyen_n_plus_1": temps_moyen_n_plus_1,
        "rapport_temps": rapport_temps
    }


base_chaine = "Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA6"
zeros_cibles = 4 # Nombre de zéros à la fin du hash
resultats = trouver_chaine_et_stats(base_chaine, zeros_cibles)
resultats


{'chaine_trouvee': 'Serano_Henri__Sara_Thibierge__Eloi_Seidlitz_DIA677031',
 'hash_trouve': '31e6f8e6377ad1c9895ef7c22b0ef9a252c0f0ec9d062178c58b416bcb470000',
 'compteur_trouve': 77031,
 'temps_moyen_n': 0.11019086837768555,
 'temps_moyen_n_plus_1': 0.3324576854705811,
 'rapport_temps': 3.0171074097634225}

## Exercice 2

1. Chiffrer un texte d'environ une demi-page (format .txt) avec AES256-CTR-PBKDF2 en utilisant un mot de passe et une valeur de compteur (IV) de 6 à 9 chiffres (sans 0).

2. Constituez un entier N en accolant le mot de passe et l'IV séparés par quatre 0.

3. Chiffrer N avec ElGamal en utilisant les nombres p, g et A fournis.

4. Sauvegarder le fichier chiffré avec l'extension .enc.

In [139]:
with open("format.txt", "r",encoding="utf-8") as file:
    text = str(file.read())
text

"Le marché des sneakers en France a connu une croissance exponentielle au cours des dernières années, devenant un phénomène de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumière ses dynamiques économiques, culturelles et sociales.\nD'un point de vue économique, le commerce de sneakers en France est stimulé par une demande croissante pour des modèles rares et exclusifs. \nLes éditions limitées et les collaborations entre marques de renom et designers célèbres créent une urgence d'achat, poussant les prix sur le marché secondaire à atteindre des sommets vertigineux. \nCette situation a engendré l'émergence d'une véritable économie parallèle, où les sneakers sont non seulement des objets de mode, mais également des actifs spéculatifs.\n\nSur le plan culturel, les sneakers occupent une place prépondérante dans l'univers de la mode et de la 

In [140]:
def generate_non_zero_number(length,password):
    return int(''.join(secrets.choice(password) for _ in range(length)))
password = "123456789"
salt = os.urandom(16)
key = pbkdf2.PBKDF2(password, salt).read(32)
iv = generate_non_zero_number(random.randint(6, 9),password)

N = int(password + "0000" + str(iv)) 
print("password: ",password)
print("IV : ",iv)
print("N : ",N)



password:  123456789
IV :  916771961
N :  1234567890000916771961


In [141]:
#On est obliger de recrée un mot de pass unique pour chiffrer le texte
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
ciphertext = aes.encrypt(text)
with open("project.enc", "wb") as encrypted_file:
    encrypted_file.write(ciphertext)
print(ciphertext)

b'A\x99\x96\xb3\xb1\xf1\x83\xf9$TL\x13V\x90\x1a\xc6\x89\x84o8\x12\x99\x0c#\x14~\xac\x90=\xd3\xb7\x1f\xe9\x1fq\xee>k\xbe\x8e\x9c\x1e\xb0\xeb\x968L\xe75\xe3V\xcc\x898;\x02\xe4i:\xce\x00\xaf)\x87\x93sY9\r\x0ep\x91RfUk\t\xa2\x01c\\1\xc5\xe0\xd0<?\x8e%\xe3\xc0u0\x12J\x0bnw0\x1f\xec\x84!\xc4\xef\xcb\xcd)\x02\x86(\xab\xb1QG\x19\x0f[^\x88Q\x94\xaa\xf9\xb3f\\\xcc\xf5\xf1\xdb\xa1\xed\x9e\x15\x06\x93\x99Z\xeb`O>\xca\x8a)}0Q\x02\x00\x1a\xb2\xf0i\xb3\xb9\xe6w\xe8t2"\rib\xec\xdc\xf7k\x93\xe1Y\x96\xf0\x84x\x89\xcem\x80\x85<e\x14C\x12\xfb\xfbT\xd3v\xc3\x8e\xa5d!"\x9d\x15\xb4\x8e\x1eyDcV,\xac\xeao\x1a\xb6\xda\xde\xa5]f\xca\\\xabZ@\nx\xf5\xa5\xba\xf1\xfd\\K\xa6\x9d\x86\xf9g_:(e\x88\x85\xdf\xc3\xdf\xe7\xcc\xab\xb6\x17p.\x06\x97\xdc\x8a\x9dj<\xcb\xda\xc2"\x14\xfa\x01\x81\xd8\xe7.\xba\xbb\xf7&b\xe6\x7fv\x1b\xfa\x86Y3\xc7 \xb5\x88R\xdf<\xdb\x8d\x02\xca\xd9\xf9<<\t\xbd\x8f\x84\x1eE\xa6O\x96\xda;\xd1\x1e\x08\x82\xc4\x94\xfd\x88K\xe2\xbc(SbX\xd2kc\xf4\xa3\x1cz\x8f\xaf\x06\xca\xc9\xcd\x82\xd4\xe9~\x00\xa9`Od\x8

In [142]:
#Et du coup de le regénéré
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext))

b"Le march\xe9 des sneakers en France a connu une croissance exponentielle au cours des derni\xe8res ann\xe9es, devenant un ph\xe9nom\xe8ne de mode et un investissement lucratif pour de nombreux amateurs et professionnels. \nCe texte explore les multiples faceteurs qui sous-tendent le commerce de sneakers dans l'Hexagone, en mettant en lumi\xe8re ses dynamiques \xe9conomiques, culturelles et sociales.\nD'un point de vue \xe9conomique, le commerce de sneakers en France est stimul\xe9 par une demande croissante pour des mod\xe8les rares et exclusifs. \nLes \xe9ditions limit\xe9es et les collaborations entre marques de renom et designers c\xe9l\xe8bres cr\xe9ent une urgence d'achat, poussant les prix sur le march\xe9 secondaire \xe0 atteindre des sommets vertigineux. \nCette situation a engendr\xe9 l'\xe9mergence d'une v\xe9ritable \xe9conomie parall\xe8le, o\xf9 les sneakers sont non seulement des objets de mode, mais \xe9galement des actifs sp\xe9culatifs.\n\nSur le plan culturel, les s

In [143]:
p = 7946851324679854613245823
g = 5
A = 7579501795988122393422986

b = random.SystemRandom().randint(2, p-2)

# Calcul de B et C pour ElGamal
B = pow(g, b, p)
K = pow(A, b, p)  # Clé secrète partagée
C = (K * N) % p  # Chiffrement de N avec la clé secrète partagée

B, C

(2276989862123912491808511, 4661322510866707316622121)

In [148]:
"""K = pow(B, g, p)

# Calculer l'inverse modulaire de K
K_inv = gm.invert(K, p)

# Récupérer N
N = (C * K_inv) % p

print(N)

password, iv = str(N)[:-13], int(str(N)[-10:])  # Ceci est un exemple; ajustez selon la structure réelle de N
print(password,iv)
# Dérivation de la clé AES à partir du mot de passe
key = pbkdf2_hmac('sha256', password.encode('utf-8'), salt, 100000, dklen=32)

aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))

# Lire le fichier chiffré
with open("project.enc", "rb") as encrypted_file:
    ciphertext = encrypted_file.read()
# Déchiffrer le contenu
aes = pyaes.AESModeOfOperationCTR(key, pyaes.Counter(iv))
print(aes.decrypt(ciphertext)) """

208


## Exercice 3

* Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).
* Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

#### 1. Démonstration de la signature ElGamal ou RSA (avec des nombres différents de ceux du cours).

In [145]:
# Fonction pour calculer l'inverse modulo
def inverse_modulo(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Fonction pour calculer la signature
def sign_elgamal(m, x, p, g):
    # Sélection d'un nombre aléatoire k
    k = random.randint(2, p - 2)
    while pow(k, -1, p - 1) == 0:  # Assure que k est premier avec p-1
        k = random.randint(2, p - 2)
    r = pow(g, k, p)
    k_inv = inverse_modulo(k, p - 1)
    s = (m - x * r) * k_inv % (p - 1)
    return r, s

# Fonction pour vérifier la signature
def verify_elgamal(m, r, s, y, p, g):
    v1 = pow(g, m, p)
    v2 = (pow(y, r, p) * pow(r, s, p)) % p
    return v1 == v2

# Paramètres ElGamal
p = 257
g = 3
x = 97

# Clé publique
y = pow(g, x, p)

# Message à signer
m = 123

# Signature
r, s = sign_elgamal(m, x, p, g)

# Vérification de la signature
valid = verify_elgamal(m, r, s, y, p, g)

print("Signature valide :", valid)


ValueError: base is not invertible for the given modulus

#### 2. Exemple d'utilisation du RSA, Diffie-Hellman, ElGamal ou de la signature ElGamal dans Python ou openssl (contexte réel).

Supposons que nous ayons un fichier texte appelé "format.txt" contenant le message à signer.



**a) Génération des clés RSA :**
Tout d'abord, nous devons générer une paire de clés RSA privée/publique à l'aide d'OpenSSL.

In [ ]:
#copy paste in a terminal
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem

.+.........+...+..+..........+........+++++++++++++++++++++++++++++++++++++++*..+.....+...+....+...+...+..+............+.+......+........+......+.......+..+.+......+...+...+.........+++++++++++++++++++++++++++++++++++++++*...+...+.+...........+................+..+......+.+........+.......+.....+.........+....+...+.....+...+.+..............+....+...+..+.+...............+..+...+.+....................+....+......+...........+....+...+.....+..........+..+......+..........+...+...............+.........+............+.....+.+........+.........+......+.........+......+.+.....+....+.........+..+...+.+..+.............+.....+.......+...+..+...+....+..+..........+........+.+......+...+..+.+........+.......+.....+..................................+......+..+...+...+.+...............+.....+...+.+......+........+...+.......+..................+.........+.........+.....++++++
.......+....+......+......+.....+...+.+..+....+......+++++++++++++++++++++++++++++++++++++++*...........+...+..+...+......+.+...

Ces commandes génèrent une clé privée private_key.pem et une clé publique correspondante `public_key.pem`.



**b) Signature du message :**

Ensuite, nous signons le message avec notre clé privée.

In [ ]:
#copy paste in a terminal
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt

Cela crée une signature du message dans le fichier signature.bin.


**c) Vérification de la signature :**

Pour vérifier la signature avec la clé publique correspondante, nous utilisons :

In [ ]:
#copy paste in a terminal
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

Verified OK


Si la vérification réussit, OpenSSL affichera "Verified OK".


Cet exemple montre comment utiliser OpenSSL pour générer une paire de clés RSA, signer un message avec la clé privée et vérifier la signature avec la clé publique. Vous pouvez répéter ces étapes dans un terminal pour expérimenter avec vos propres messages et clés.

In [ ]:
!openssl genpkey -algorithm RSA -out private_key.pem
!openssl rsa -pubout -in private_key.pem -out public_key.pem
!openssl dgst -sha256 -sign private_key.pem -out signature.bin format.txt       
!openssl dgst -sha256 -verify public_key.pem -signature signature.bin format.txt

........+.....+++++++++++++++++++++++++++++++++++++++*....+..+.+...........+......+...+.+...........+.......+..+...............+.+...+..+............+...+...+.......+...+.........+..+.+++++++++++++++++++++++++++++++++++++++*..+..........+..+.+............+.....+...+.........+.........+.......+..++++++
.......+........+.+.....+....+...+.....+.......+...+..+......+.+++++++++++++++++++++++++++++++++++++++*......+......+...+....+...+..+............+.........+.......+...+++++++++++++++++++++++++++++++++++++++*......+...+......+.+....................+..........+...........+.........+.+..+.+...........................+...+.........+..+.+...+.....+..........+..+...+......+......+.........+....+...............+.........+......+..+....+...+...+............+.....+..................+...+....+...+.........+.....+.+.....+...+....+............+.........+..+....+......+..+.......+.....+...............+...+.+......+.....+.+..+...+.........+...................+..+............+......+...............+....

Verified OK
